## Results Section 6.1: Hyperparameter optimization
In this notebook, we outline the post-processing procedure for the hyperparameter optimization procedure. We first find the optimal parameters and then reproduce the results presented in Table 2.

### Imports and setup

In [ ]:
import pandas as pd
import numpy as np
import os
import json

In [ ]:
source_file_optimization = "../../data/optimization.csv"
source_output = '../../data/output'
output_folder = "../../data/paper"

os.makedirs(os.path.join(output_folder, '61'), exist_ok=True)

### Read optimization file
We run the optimization file containing the runtimes for each matrix and post-process the 'name' column which gives us all information on the parameter used.

In [ ]:
csv = pd.read_csv(source_file_optimization, sep="\t", header=None)
csv.columns = ["name", "time", "std", "runs", "success", "real_success"]
csv["qbs"] = csv["name"].apply(lambda x: int(x.split("/")[3]))
csv["parameter"] = csv["name"].apply(lambda x: x.split("/")[4])
csv["value"] = csv["name"].apply(lambda x: x.split("/")[5] if not x.split("/")[5].endswith("txt") else 0)

csv["value"] = csv["value"].astype(float)
csv["matrix"] = csv["name"].apply(lambda x: x.split("/")[6] if len(x.split("/")) == 8 else x.split("/")[5])
csv["type"] = csv["matrix"].apply(lambda x: x.split("_")[1])

We change the average time if the real success is not 100, since the time then does not include the time past the last successful run


In [ ]:
should_add = csv["real_success"] < 100
numbers = np.maximum(np.array(csv["real_success"]), 1)
csv["time"] = csv["time"] + should_add * ((600 - csv["time"] * csv["real_success"]) / numbers)

### Get the best result in absolute time
We get the best values for the parameters in absolute time. We will use this later as the baseline parameter for the optimization in relative time (which is the actual value that we care about, giving each matrix the same weight)

In [ ]:
best_values = csv.groupby(["parameter", "value"]).mean(numeric_only=True)["time"].reset_index()
best_times = best_values.groupby("parameter").min()["time"].reset_index()
best_values = best_values.merge(best_times, on=["parameter", "time"], how="inner")

best_values

Now, we run the optimization process for relative time, usually giving the same result as for the absolute variant, but sometimes differing

In [ ]:
all_csv_parameters = None
for parameter in best_values["parameter"].unique():
    csv_parameter = csv[csv["parameter"] == parameter].copy()
    csv_for_merging = csv_parameter[['matrix', 'qbs', 'value', "time"]].copy()
    # merge the time of the best value on each matrix
    parameter_value = best_values[best_values["parameter"] == parameter]["value"].values[0]
    csv_parameter = csv_parameter.merge(csv_for_merging[csv_for_merging["value"] == parameter_value], on=["matrix", "qbs"], how="inner")
    if all_csv_parameters is None:
        all_csv_parameters = csv_parameter
    else:
        all_csv_parameters = pd.concat([all_csv_parameters, csv_parameter])

all_csv_parameters["relative"] = all_csv_parameters["time_x"] / all_csv_parameters["time_y"]

grouped = all_csv_parameters[["relative", "parameter", "value_x"]].groupby(["parameter", "value_x"]).mean()
output = dict()
# get value optimal for each parameter
for parameter in grouped.index.get_level_values(0).unique():
    # set parameter equal to the value that has the lowest relative time
    output[parameter] = grouped.loc[parameter].idxmin().values[0]

We convert the names to those used in the paper and only include the parameters we actually optimized using this procedure.

In [ ]:
naming_conversion = {
    'n-norm': r'n_{\text{norm}}',
    'pid': r'P_{\text{Id}}',
    'start-temp': 'T(0)',
    'n-start-gates': r'\ell',
    'iterations-factor': r'n_{\text{step}} / (n\ell)',
}

output = {naming_conversion[k]: v for k, v in output.items() if k in naming_conversion.keys()}
os.makedirs(os.path.join(output_folder, '61'), exist_ok=True)
json.dump(output, open(os.path.join(output_folder, "61", "parameters.json"), "w"), indent=4)
output

### Results in Table 2
We determine the average speedup provided by the normal Synthetiq versus Synthetiq with some parameter changes

In [ ]:
csv_only_table2 = csv[csv["parameter"].isin(["", "no-resynth", "simple", "no-perms"])] 
csv_normal = csv_only_table2[csv_only_table2["parameter"] == ""]
csv_normal = csv_normal[["matrix", "qbs", "time"]]
csv_only_merged = csv_only_table2.merge(csv_normal, on=["matrix", "qbs"], how="inner")
csv_only_merged["relative"] = csv_only_merged["time_x"] / csv_only_merged["time_y"] 

In [ ]:
csv_only_merged = csv_only_merged[csv_only_merged["parameter"] != ""]
grouped = csv_only_merged[["relative", "parameter", "type"]].groupby(["parameter", "type"]).mean()
grouped.to_csv(os.path.join(output_folder, '61', "table2.csv"))
grouped